In [169]:
# Find project root
import subprocess, os
cmd = 'git rev-parse --show-toplevel'
root = subprocess.check_output(cmd, shell=True, text=True)
os.chdir(root.strip())
print('CWD:', os.getcwd())

CWD: C:\Users\AndBondStyle\Projects\lab-cartpole


In [170]:
from sessions.collector import SessionData
from misc.simple_server import session_widget
from pathlib import Path
import pandas
import struct
import json
import math

PATH = Path('data/sessions/v3_lqr')
DATA = SessionData.load(PATH / 'session.json')

In [171]:
DATA.values

{'state.position': SessionData.Value(id='state.position', name='state.position', unit='?'),
 'state.velocity': SessionData.Value(id='state.velocity', name='state.velocity', unit='?'),
 'state.acceleration': SessionData.Value(id='state.acceleration', name='state.acceleration', unit='?'),
 'state.pole_angle': SessionData.Value(id='state.pole_angle', name='state.pole_angle', unit='?'),
 'state.pole_angular_velocity': SessionData.Value(id='state.pole_angular_velocity', name='state.pole_angular_velocity', unit='?'),
 'state.error_code': SessionData.Value(id='state.error_code', name='state.error_code', unit='?'),
 'state.accelerometer_value': SessionData.Value(id='state.accelerometer_value', name='state.accelerometer_value', unit='?'),
 'state.motor_angle': SessionData.Value(id='state.motor_angle', name='state.motor_angle', unit='?'),
 'state.motor_velocity': SessionData.Value(id='state.motor_velocity', name='state.motor_velocity', unit='?'),
 'target.acceleration': SessionData.Value(id='tar

In [172]:
serial_tx_data = pandas.read_csv(
    PATH / '2_async_serial.csv',
    header=0,
    names=['time', 'value', 'parity_err', 'framing_err'],
)
serial_tx_data['time'] *= 1_000_000
analyzer_offset = serial_tx_data.iloc[0]['time'] - 2_000_000
serial_tx_data['time'] -= analyzer_offset
print('Analyzer time offset:', analyzer_offset, 'us')
serial_tx_data

Analyzer time offset: -1948985.5 us


,time,value,parity_err,framing_err
0,2000000.0,0x67,NaN,NaN
1,2000086.5,0x65,NaN,NaN
2,2000173.5,0x74,NaN,NaN
3,2000260.5,0x20,NaN,NaN
4,2000347.5,0x73,NaN,NaN
...,...,...,...,...
4028,6714650.5,0x6F,NaN,NaN
4029,6714737.5,0x72,NaN,NaN
4030,6714824.5,0x5F,NaN,NaN
4031,6714911.5,0x76,NaN,NaN


In [173]:
i2c_data = pandas.read_csv(
    PATH / '3_i2c.csv',
    header=0,
    names=['time', 'packet_id', 'address', 'data', 'read_write', 'ack_nak'],
)
i2c_data['time'] = i2c_data['time'] * 1_000_000 - analyzer_offset
i2c_data

,time,packet_id,address,data,read_write,ack_nak
0,1950169.0,0,0x6C,0x0C,Write,ACK
1,1950426.0,1,0x6D,0x07,Read,NAK
2,1950685.0,2,0x6C,0x0D,Write,ACK
3,1950942.5,3,0x6D,0xBA,Read,NAK
4,1952339.5,4,0x6C,0x0C,Write,ACK
...,...,...,...,...,...,...
8211,6778867.5,8211,0x6D,0x64,Read,NAK
8212,6780257.5,8212,0x6C,0x0C,Write,ACK
8213,6780514.0,8213,0x6D,0x00,Read,NAK
8214,6780779.5,8214,0x6C,0x0D,Write,ACK


In [174]:
enc_address = 0x36  # I2C encoder address
msb_address = 0x0C  # Raw angle MSB register address
lsb_address = 0x0D  # Raw angle LSB register address
zero_angle = 2.9406 # Const from encoder constructor
reverse = True      # ... from firmware

def convert_raw_angle(raw_angle):
    angle = raw_angle / 4096 * 2 * math.pi - zero_angle
    if angle < 0: angle += 2 * math.pi
    if reverse: angle = 2 * math.pi - angle
    return angle

curr_mode = 'write'
curr_address = msb_address
raw_angle = 0
value_id ='analyzer.encoder_angle'
hw_encoder_angle = DATA.values[value_id] = SessionData.Value(value_id, value_id, 'rad')
for _, row in i2c_data.iterrows():
    if row.time < 0: continue
    address = int(row.address, 16)
    data = int(row.data, 16)
    mode = row.read_write.lower()
    # print(hex(address), hex(data), mode)
    if address >> 1 != enc_address:
        print('!')
        continue
    if mode != curr_mode:
        print('!')
        continue
        
#     if abs(row.time - 7.212 * 1_000_000) < 0.05 * 1_000_000:
#         print((row.time + analyzer_offset) / 1_000_000)
#         print(curr_mode, hex(curr_address), bin(raw_angle), data == curr_address)
    
    # print(curr_mode, hex(curr_address), bin(raw_angle), data == curr_address)
    if mode == 'write' and data == curr_address:
        curr_mode = 'read'
    elif mode == 'read' and curr_address == msb_address:
        raw_angle = data << 8
        curr_address = lsb_address
        curr_mode = 'write'
    elif mode == 'read' and curr_address == lsb_address:
        raw_angle |= data
        angle = convert_raw_angle(raw_angle)
        hw_encoder_angle.x.append(int(row.time))
        hw_encoder_angle.y.append(angle)
        # print('*', bin(raw_angle), raw_angle, angle)
        curr_address = msb_address
        curr_mode = 'write'
    else:
        print('!')
        
    #if len(raw_angle_list) > 10: break

print('Number of parsed values:', len(hw_encoder_angle.x))

Number of parsed values: 2054


In [175]:
def convert_and_print(hex_value):
    angle = convert_raw_angle(hex_value)
    print(f'{angle:.5f} {hex_value:04x} {hex_value:016b} {hex_value}')
    return angle

for i in range(16):
    mask = 1 << i
    print('MASK:', bin(mask))
    a = convert_and_print(0x0ff3)
    convert_and_print(0x0f02)  # <-- anomaly
    x = convert_and_print(0x0f02 ^ mask)  # <-- anomaly fix
    b = convert_and_print(0x0023)
    print(a >= x >= b)
    print()
    a = convert_and_print(0x00DE)
    convert_and_print(0x0002)  # <-- anomaly
    x = convert_and_print(0x0002 ^ mask)  # <-- anomaly fix
    b = convert_and_print(0x0111)
    print(a >= x >= b)
    print()

MASK: 0b1
2.96054 0ff3 0000111111110011 4083
3.33023 0f02 0000111100000010 3842
3.32870 0f03 0000111100000011 3843
2.88691 0023 0000000000100011 35
False

2.60006 00de 0000000011011110 222
2.93753 0002 0000000000000010 2
2.93600 0003 0000000000000011 3
2.52182 0111 0000000100010001 273
False

MASK: 0b10
2.96054 0ff3 0000111111110011 4083
3.33023 0f02 0000111100000010 3842
3.33330 0f00 0000111100000000 3840
2.88691 0023 0000000000100011 35
False

2.60006 00de 0000000011011110 222
2.93753 0002 0000000000000010 2
2.94060 0000 0000000000000000 0
2.52182 0111 0000000100010001 273
False

MASK: 0b100
2.96054 0ff3 0000111111110011 4083
3.33023 0f02 0000111100000010 3842
3.32410 0f06 0000111100000110 3846
2.88691 0023 0000000000100011 35
False

2.60006 00de 0000000011011110 222
2.93753 0002 0000000000000010 2
2.93140 0006 0000000000000110 6
2.52182 0111 0000000100010001 273
False

MASK: 0b1000
2.96054 0ff3 0000111111110011 4083
3.33023 0f02 0000111100000010 3842
3.31796 0f0a 0000111100001010 38

In [176]:
(5.32 - 5.68) / 2 / math.pi * 360

-20.626480624709604

In [177]:
abs_pole_angles = []
rotations_arr = []
rotations = 0
prev = 0
max_delta = math.pi
for curr in DATA.values['state.pole_angle'].y:
    delta = curr - prev
    if delta > max_delta:
        rotations -= 1
    elif delta < -max_delta:
        rotations += 1
    abs_angle = 2 * math.pi * rotations + curr
    abs_pole_angles.append(abs_angle)
    rotations_arr.append(2 * math.pi * rotations)
    prev = curr
    
id = 'abs_pole_angle'
DATA.values[id] = SessionData.Value(id, id, 'rad', DATA.values['state.pole_angle'].x, abs_pole_angles)
id = 'pole_rotations'
DATA.values[id] = SessionData.Value(id, id, 'rad', DATA.values['state.pole_angle'].x, rotations_arr)

In [178]:
DATA.groups = [
#     SessionData.Group(name='Position', values=['state.position']),
#     SessionData.Group(name='Velocity', values=['state.velocity']),
#     SessionData.Group(name='Acceleration', values=['state.acceleration', 'target.acceleration']),
    SessionData.Group(name='Pole angle', values=['state.pole_angle', 'analyzer.encoder_angle']),
    SessionData.Group(name='Absolute pole angle', values=['abs_pole_angle', 'pole_rotations']),
    SessionData.Group(name='Pole velocity', values=['state.pole_angular_velocity']),
]
session_widget(DATA)

IFrame URL: http://localhost:63027/?id=_


In [179]:
t1 = 6.721909
a1 = 5.221629
t2 = 6.728088
a2 = 4.908697

da = a2 - a1
print('delta angle:', da)
dt = t2 - t1
print('delta time:', dt)
if abs(da) > 1.8 * math.pi:
    print('fixing rotation')
    da -= (2 * math.pi) if da > 0 else (-2 * math.pi)
print('delta angle out:', da)
vel = da / dt
print('velocity:', vel)
if abs(vel) > 10 * 2 * math.pi:
    print('spike!')
print(5 * 2 * math.pi)

delta angle: -0.312932
delta time: 0.0061789999999994905
delta angle out: -0.312932
velocity: -50.64444084803784
31.41592653589793
